In [1]:
import pandas as pd
import bayes

In [2]:
data = pd.read_csv(r"C:\Users\Muc\Downloads\3DCi_1611_2111.csv")
data.loc[:,'day0_date'] = pd.to_datetime(data.loc[:,'day0_date'])
data.loc[:,'act_date'] = pd.to_datetime(data.loc[:,'act_date'])
print(data.app_version.value_counts())
print(data.experiment_group.value_counts())

1.0.4    12538
1.0.2       36
1.0.3       13
Name: app_version, dtype: int64
firebase_exp_1_group0    6171
firebase_exp_1_group1    6147
Name: experiment_group, dtype: int64


In [3]:
from datetime import timedelta

def preprocess(data, metrics, ab_list, compare_by,  day_window = 3, **kwargs):
    data.loc[:,'day0_date'] = pd.to_datetime(data.loc[:,'day0_date'])
    data.loc[:,'act_date'] = pd.to_datetime(data.loc[:,'act_date'])
    if (data.act_date.max() - data.day0_date.min()) > timedelta(3):
        data = data[data.loc[:,'act_date'] < data.loc[:,'act_date'].max()] #drop max act_date 
        data = data[data.loc[:,'day0_date'] <= data.loc[:,'act_date'].max()-timedelta(days=day_window+1)] #find range day0_date
        data = data[(data.loc[:,'day_diff'] <= day_window) & (data.loc[:,'day_diff']>= 0)]

    for key, value in kwargs.items():
        data = data[data.loc[:,key] == value]
    
    if compare_by == 'app_version':
        data = data.query('app_version in @ab_list')
    elif compare_by == 'experiment_group':
        data = data.query('experiment_group in @ab_list')
    
    data['combined_ad'] = data['rv_imp_sum'] * 1.3 + data['is_imp_sum']
    dim = data.groupby('resettable_device_id_or_app_instance_id')[compare_by].max().reset_index()
    fact = data.groupby('resettable_device_id_or_app_instance_id')[metrics].sum()
    data = dim.merge(fact, on ='resettable_device_id_or_app_instance_id')
    return data


In [4]:
metrics = [
    'time_in_game_sum'
    ,'rv_imp_sum'
    ,'is_imp_sum'
    ,'combined_ad'
    ,'battle_play_sum'
    ,'battle_end_sum'
    ,'ad_value_sum'
    # ,'user_value_sum'
]

filter = {
    'country':'United States'
    # ,'event_segment': 'app_update--FALSE'
    # ,'app_version' : '1.1.6'
    # ,'media_source' : "Organic"
}
# ab_list = ['1.0.7', '1.0.9']
ab_list = ['firebase_exp_1_group0', 'firebase_exp_1_group1']
compare_by = 'experiment_group'
day_window = 3


from_date = data.day0_date.min().strftime('%d/%m')
to_date = (data.day0_date.max() - timedelta(days = day_window + 1)).strftime('%d/%m')
data = preprocess(data, metrics, ab_list, compare_by = compare_by, day_window = day_window, **filter)
descriptive_metric = []
# number_user = data.groupby('app_version').agg({'resettable_device_id_or_app_instance_id':pd.Series.nunique})
number_user = data.groupby('experiment_group').agg({'resettable_device_id_or_app_instance_id':pd.Series.nunique})


for i in metrics:
    descriptive_metric.append(data.groupby('experiment_group').agg({i: ['mean', 'std']}))
descriptive_metric = pd.concat(descriptive_metric, axis=1)
descriptive_metric
for metric in metrics:
    data_metrics = descriptive_metric[metric]
    na, nb = number_user.resettable_device_id_or_app_instance_id
    ma, mb = data_metrics['mean']
    sa, sb = data_metrics['std']
    globals()[metric] = bayes.gaussian_ab_test(m_a=ma, s_a=sa, n_a=na, m_b=mb, s_b=sb, n_b=nb)

C:\Users\Muc\AppData\Local\Temp/ipykernel_18064/3199708377.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['combined_ad'] = data['rv_imp_sum'] * 1.3 + data['is_imp_sum']


In [5]:
import xlsxwriter
from datetime import date
from pathlib import Path

project = '3DCi_test'
path = 'C:/Users/Muc/Desktop/report/{}'.format(project)
Path(path).mkdir(parents=True, exist_ok=True)
# filename = 'report_{}_{}'.format(date.today(), project)
workbook = xlsxwriter.Workbook('{}/report_{}_{}.xlsx'.format(path, date.today(), project))
worksheet = workbook.add_worksheet("Report")


title_format = workbook.add_format({'bold':True, 'font_size':14})
worksheet.write('A1', 'AB TEST', title_format)
worksheet.write('B1', project, title_format)

# -----------------------------------
subtitle_format = workbook.add_format({'font_size':11})

worksheet.write('A2', 'country', subtitle_format)
worksheet.write('B2', filter['country'], subtitle_format)

worksheet.write('A3', 'media_source', subtitle_format)
worksheet.write('B3', 'All', subtitle_format)

worksheet.write('A3', 'install date', subtitle_format)
worksheet.write('B3', from_date + '-' + to_date, subtitle_format)

# ------------------------------------------
format_columns =  workbook.add_format({'font_size':11, 'bold':True, 'align': 'center', 'valign': 'vcenter' })

worksheet.merge_range("A6:A7", 'Metrics', format_columns)
worksheet.merge_range("B6:B7", 'Mất mát dự tính nếu chọn: {}'.format(ab_list[1]), format_columns)
worksheet.write('C6', 'Baseline: {}'.format(ab_list[0]), format_columns)
worksheet.write('C7', 'Value', format_columns)

worksheet.merge_range("D6:E6", 'Variation: {}'.format(ab_list[1]), format_columns)
worksheet.write('D7', 'Value', format_columns)
worksheet.write('E7', 'Xác suất tốt hơn baseline', format_columns)

worksheet.write('B8', 'Số Users (new)', format_columns)
worksheet.write('C8', na , format_columns)
worksheet.write('D8', nb , format_columns)

# ------------------------------------
format_rows = workbook.add_format({'font_size':11, 'align': 'center', 'valign': 'vcenter' })
row = 8
col = 0
for i in metrics:
    worksheet.merge_range(row, col, row+1, col, i , format_columns)
    
    metric = eval(i)
    # col+=1
    worksheet.write(row+1, col+1, '{} / user'.format(round(metric['risk'][1],2)), format_columns)
    worksheet.write(row , col+1 , '{:.2%}'.format(metric['risk'][1] / metric['mean']['m_b'] ), format_columns)
    worksheet.merge_range(row, col+2, row+1, col+2, '{:.2f}'.format(metric['mean']['m_a']), format_rows)
    worksheet.merge_range(row, col+3, row+1, col+3, '{:.2f}'.format(metric['mean']['m_b']), format_rows)
    worksheet.merge_range(row, col+4, row+1, col+4, '{:.2%}'.format(metric['chance_to_win']), format_columns)    

    row+=2

 
workbook.close()

In [63]:
def preprocess(data, day_window = 3, **kwargs):
    #cast day0date and act date to date
    data.loc[:,'day0_date'] = pd.to_datetime(data.loc[:,'day0_date'])
    data.loc[:,'act_date'] = pd.to_datetime(data.loc[:,'act_date'])
    if (data.act_date.max() - data.day0_date.min()) > timedelta(3):
        data = data[data.loc[:,'act_date'] < data.loc[:,'act_date'].max()] #drop max act_date 
        data = data[data.loc[:,'day0_date'] <= data.loc[:,'act_date'].max()-timedelta(days=day_window)] #find range day0_date
        data = data[(data.loc[:,'day_diff'] <= day_window) & (data.loc[:,'day_diff']>= 0)]

    for key, value in kwargs.items():
        data = data[data.loc[:,key] == value]
    
    data['combined_ad'] = data['rv_imp_sum'] * 1.3 + data['is_imp_sum']
    return data

def get_abtest_data(df, target, **kwargs):
    ab_data = preprocess(df, **kwargs)
    ab_data = ab_data.groupby('resettable_device_id_or_app_instance_id')[target].sum().reset_index(drop=True)
    return ab_data

def gameinfo(df, METRICS, **kwargs):
    data = preprocess(df, day_window = 3, **kwargs)
    data_by_day_diff = data.groupby('day_diff')[METRICS].sum()
    data_by_day_diff.loc['Grand Total'] = data_by_day_diff.iloc[:,1:].sum()

    data_per_dau = data_by_day_diff.copy()
    data_per_dau.iloc[:-1,1:] = data_per_dau.iloc[:-1,1:].div(data_per_dau.iloc[:-1,0], axis=0)
    data_per_dau.iloc[-1,1:] = data_per_dau.iloc[-1,1:]/data_per_dau.iloc[0,0]
    data_per_dau['dau'] = data_per_dau['dau']/data_per_dau.iloc[0,0]
    data_per_dau = data_per_dau.style.format({'dau': "{:.2%}"})
    return data_by_day_diff

In [65]:
a = {
'country' : 'United States'

    # ,'app_version' : '1.0.7' 
    ,'experiment_group':'firebase_exp_1_group0'
    # ,'media_source' : "Organic"                                       
    # ,'event_segment' : 'app_update--FALSE' 
    }                                                                   
b = {
    'country' : 'United States'
    # ,'app_version' : '1.0.9' 
    ,'experiment_group':'firebase_exp_1_group1'
    # ,'media_source' : 'Organic'
    # ,'event_segment' : 'app_update--FALSE' 
    }
metrics =  ['dau','time_in_game_sum','rv_imp_sum', 'is_imp_sum', 'combined_ad', 'battle_play_sum', 'battle_end_sum']
pd.DataFrame(gameinfo(df, metrics, **b))

,dau,time_in_game_sum,rv_imp_sum,is_imp_sum,combined_ad,battle_play_sum,battle_end_sum
day_diff,,,,,,,
0,1864.0,1146689.464,5840.0,4418.0,12010.0,8880.0,7409.0
1,594.0,419788.030,1444.0,1429.0,3306.2,3242.0,2797.0
2,370.0,254662.388,829.0,816.0,1893.7,2082.0,1791.0
3,263.0,131796.187,505.0,442.0,1098.5,1002.0,804.0
Grand Total,NaN,1952936.069,8618.0,7105.0,18308.4,15206.0,12801.0
